In this notebook we are going to construct a simple knowledge graph using Python, and run some queries on the graph in Neo4j.

If you would like to run this code yourself, you will need to install the `py2neo` package in Python 3.

To run part 3 onwards, you will need to install Neo4j, which can be downloaded at https://neo4j.com/download/.

I will be running through the code during part 2 of the master class so there is no need to install anything unless you would also like to try the code out yourself and run some graph queries.

# 1. Read in the data

Before we can build a graph, we must first read in the example datasets:

- `work_order_file`: A csv file containing a set of work orders.

Here is an example of what the first few rows of each dataset look like:

![alt text](images/example-data.png "Example datasets")

TODO: Remove downtime events completely

We are using the simple `csv` library to read in the data, though this can also be done using `pandas`.

# Installing required packages

To run this notebook you will need to install the following via pip
:

In [ ]:
!pip install flair

# Inspecting the data

Let's start by inspecting the CSV dataset.

In [1]:
from csv import DictReader

work_order_file = "data/sample_work_orders.csv"

# A simple function to read in a csv file and return a list,
# where each element in the list is a dictionary of {heading : value}
def load_csv(filename):
    data = []
    with open(filename, 'r') as f:
        reader = DictReader(f)
        for row in reader:
            data.append(row)
    return data

        
work_order_data = load_csv(work_order_file)

for row in work_order_data:
    print(row)

    


OrderedDict([('StartDate', '10/07/2005'), ('FLOC', '1234.1.1'), ('ShortText', 'repair cracked hyd tank')])
OrderedDict([('StartDate', '14/07/2005'), ('FLOC', '1234.1.2'), ('ShortText', 'engine wont start')])
OrderedDict([('StartDate', '17/07/2005'), ('FLOC', '1234.1.3'), ('ShortText', 'a/c blowing hot air')])
OrderedDict([('StartDate', '20/07/2005'), ('FLOC', '1234.1.2'), ('ShortText', 'engin u/s')])
OrderedDict([('StartDate', '21/07/2005'), ('FLOC', '1234.1.2'), ('ShortText', 'fix engine')])
OrderedDict([('StartDate', '22/07/2005'), ('FLOC', '1234.1.4'), ('ShortText', 'pump service')])
OrderedDict([('StartDate', '23/07/2005'), ('FLOC', '1234.1.4'), ('ShortText', 'pump leak')])
OrderedDict([('StartDate', '24/07/2005'), ('FLOC', '1234.1.4'), ('ShortText', 'fix leak on pump')])
OrderedDict([('StartDate', '25/07/2005'), ('FLOC', '1234.1.2'), ('ShortText', 'engine not running')])
OrderedDict([('StartDate', '26/07/2005'), ('FLOC', '1234.1.2'), ('ShortText', 'engine has problems starting')])

# 2. Named Entity Recognition

Our first task is to extract the entities in the short text descriptions and construct nodes from those entities. This is how we are able to unlock the knowledge captured within the short text and combine it with the structured fields.

![alt text](images/extracting-entities-v2.png "Extracting entities")

## 2.1. Inspecting the data

We can also inspect the dataset we will use to train our model.

In [53]:
NER_DATASETS_PATH = "data/ner_dataset"


def to_conll_document(s: str):
    """Create a ConllDocument from a string as it appears
    in a Conll-formatted file.

    Args:
        s (str): A string, separated by newlines, where each
        line is a token, then a comma and space, then a label.

    Returns:
        ConllDocument: A ConllDocument created from s.
    """
    tokens, labels = [], []
    for line in s.split("\n"):
        if len(line.strip()) == 0:
            continue
        token, label = line.split()

        tokens.append(token)
        labels.append(label)
    return {'tokens': tokens, 'labels': labels}


def load_conll_dataset(filename: str) -> list:
    """Load a list of documents from the given CONLL-formatted dataset.

    Args:
        filename (str): The filename to load from.

    Returns:
        list: A list of documents.
    """
    documents = []
    with open(filename, "r") as f:
        docs = f.read().split("\n\n")
        for d in docs:
            if len(d) == 0:
                continue
            document = to_conll_document(d)
            documents.append(document)
    print(f"Loaded {len(documents)} documents from {filename}.")
    return documents

train_dataset = load_conll_dataset(os.path.join(NER_DATASETS_PATH, 'train.txt'))

print(train_dataset[0])

Loaded 3200 documents from data/ner_dataset\train.txt.
{'tokens': ['ram', 'on', 'cup', 'rod', 'support', 'broken'], 'labels': ['B-Item', 'B-Location', 'B-Item', 'B-Item', 'I-Item', 'B-Observation']}


In [ ]:
# TODO: Write code to inspect the training data (CONLL format)

## 2.2 Define an abstract base class for NER Models

(explanation about ABCs and the four methods we are going to use in all our NER models)

In [7]:
""" Abstract base class for the NER Model. """

from abc import ABC, abstractmethod


class NERModel(ABC):
    def __init__(self):
        pass

    @abstractmethod
    def train(self, conll_datasets_path: str):
        pass

    @abstractmethod
    def inference(self, dataset_filename: str):
        pass

    @abstractmethod
    def load(self, model_path):
        pass

    @abstractmethod
    def save(self, model_path):
        pass


## 2.3 Define a Flair-based NER Model class

In this tutorial we will use [Flair](https://github.com/flairNLP/flair), which simplifies the process of building a deep learning model for a variety of NLP tasks.

The code below is a class representing a `FlairNERModel`, which is based on the `NERModel` class above. It has the same four methods, i.e `train()`, `inference()`, `load()`, and `save()`.

In [55]:
"""A Flair-based Named Entity Recognition model. Learns to predict entity
classes via deep learning."""


# TODO: Tidy up, fix this code as it does not work atm in this notebook


import os
import flair
from flair.data import Corpus, Sentence
from flair.datasets import ColumnCorpus
from flair.embeddings import (
    StackedEmbeddings,
    FlairEmbeddings,
)
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer
from typing import List
from flair.visual.training_curves import Plotter
import torch


from .NERModel import NERModel

# TODO: Get rid of ConllDataset/ConllDocument and just use lists
from mwo2kg_datasets import (
    ConllDataset,
    ConllDocument,
)

HIDDEN_SIZE = 256

# Check whether CUDA is available and set the device accordingly
if torch.cuda.is_available():
    flair.device = torch.device("cuda:0")
else:
    flair.device = torch.device("cpu")
print("Device:", flair.device)


class FlairNERModel(NERModel):

    model_name: str = "Flair"

    """A Flair-based Named Entity Recognition model.
    """

    def __init__(self):
        super(FlairNERModel, self).__init__()

        self.model = None

    def train(self, datasets_path: os.path, trained_model_path: os.path):
        """ Train the Flair model on the given conll datasets.

        Args:
            datasets_path (os.path): The folder containing the
              train, dev and text CONLL-formatted datasets.
            trained_model_path (os.path): The folder to save the trained
              model to.
        """

        columns = {0: "text", 1: "ner"}
        corpus: Corpus = ColumnCorpus(
            datasets_path,
            columns,
            train_file="train.txt",
            dev_file="dev.txt",
            test_file="test.txt",
        )
        label_dict = corpus.make_label_dictionary(label_type="ner")

        # Train the sequence tagger
        embedding_types = [
            FlairEmbeddings("mix-forward"),
            FlairEmbeddings("mix-backward"),
        ]

        embeddings = StackedEmbeddings(embeddings=embedding_types)

        tagger = SequenceTagger(
            hidden_size=HIDDEN_SIZE,
            embeddings=embeddings,
            tag_dictionary=label_dict,
            tag_type="ner",
            use_crf=True,
        )

        trainer = ModelTrainer(tagger, corpus)

        sm = "cpu"
        if torch.cuda.is_available():
            sm = "gpu"
        trainer.train(
            trained_model_path,
            learning_rate=0.1,
            mini_batch_size=32,
            max_epochs=10,
            embeddings_storage_mode=sm,
        )

        plotter = Plotter()
        plotter.plot_weights(os.path.join(trained_model_path, "weights.txt"))

        self.load(trained_model_path)

    def inference(self, raw_sents: list) -> ConllDataset:
        """Run the inference on a given list of short texts.

        Args:
            raw_sents (list): The list of raw sents to run the inference on.

        Returns:
            ConllDataset: The ConllDataset of preds.

        Raises:
            ValueError: If the model has not yet been trained.
        """
        if self.model is None:
            raise ValueError(
                "The KGC Model has not yet been trained. "
                "Please train this Flair model before proceeding."
            )

        preds_dataset = ConllDataset()

        for i, tokens in enumerate(raw_sents):
            labels = self._tag_sentence(tokens)
            doc = ConllDocument(tokens, labels)
            preds_dataset.add_document(doc)

        return preds_dataset

    def load(self, model_path: os.path):
        """Load the model from the specified path.

        Args:
            model_path (os.path): The path to load.

        Raises:
            ValueError: If the path does not exist i.e. model not yet trained.
        """
        model_file = os.path.join(model_path, "final-model.pt")
        if not os.path.exists(model_file):
            raise ValueError(
                "The NER Model has not yet been trained (the Flair resource "
                "files are missing)."
            )
        self.model = SequenceTagger.load(model_file)

    def save(self, model_path):
        """Flair model does not need a save function -
        it saves after training."""
        raise NotImplementedError

    def _tag_sentence(self, sentence: List[str]) -> List[str]:
        """Tag the given sentence (list of tokens) via the model.

        Args:
            sentence (List[str]): A list of tokens.

        Returns:
            List[str]: A list of labels.
        """
        sentence_obj = Sentence(sentence, use_tokenizer=False)
        self.model.predict(sentence_obj)
        labels = ["O"] * len(sentence)

        for entity in sentence_obj.get_spans("ner"):
            for i, token in enumerate(entity):
                label = entity.get_label("ner").value
                prefix = "B-" if i == 0 else "I-"
                
                # Token idx starts from 1 in Flair.
                labels[token.idx - 1] = prefix + label

        return labels

ModuleNotFoundError: No module named 'flair'

### (optional) Define a DictionaryNERModel class

If for some reason you are not able to use the Flair library (not enough space, computer not powerful enough etc), here is a simple model you can use to extract the entities, albeit with a much weaker performance. This one scans the training data, builds a mapping between each phrase (one or more tokens in a row) and the most common entity type associated with that phrase, then uses that entity type as the prediction when seeing that token in the test data.

#### TODO: Move this into a separate python script and just import it via `import DictionaryNERModel`. No point having it here

In [128]:
""" A dictionary-based NER model. Can be used as an alternative
to Flair, which is cumbersome to run and install."""

import os
import json
import pickle as pkl


class DictionaryNERModel2183(NERModel):

    """The dictionary-based NER model. Labels sentences based on
    the most frequent label assigned to each phrase as per the
    training dataset.

    Attributes:
        _chunked_frequency_dict (dict): A dict to keep track of the
          most frequent label for each phrase, for each phrase length.
    """

    model_name: str = "Dictionary-based"

    def __init__(self):
        super(DictionaryNERModel2183, self).__init__()
        self._chunked_frequency_dict = None

    def train(self, datasets_path: os.path, trained_model_path: os.path):
        """ "Train" the dictionary model on the given conll datasets.
        Note it isn't actually training... just building a dictionary
        from the training/dev datasets and using it as a means to
        heuristically tag the test sents.

        Args:
            datasets_path (os.path): The folder containing the
              train, dev and text CONLL-formatted datasets.
            trained_model_path (os.path): The folder to store the
              'trained model' i.e. the freq dict etc.
        """
        print("Building dictionary...")
        conll_dataset = self._load_conll_data(datasets_path)
        frequency_dict = self._build_frequency_dict(conll_dataset)

        self._chunked_frequency_dict = self._chunk_frequency_dict(
            frequency_dict
        )

        self.save(trained_model_path)

    def load(self, model_path):
        """Load the chunked frequency dict from the given folder.

        Args:
            model_path (str): The filename containing the chunked
              frequency dict (pickle file).

        Raises:
            ValueError: If the model.pkl file is missing, i.e.
              model has not been trained.
        """
        trained_model_path = os.path.join(model_path, "model.pkl")
        if not os.path.exists(trained_model_path) or not os.path.exists(
            model_path
        ):
            raise ValueError(
                "The KGC Model has not yet been trained (the model.pkl"
                " file is missing)."
            )

        with open(trained_model_path, "rb") as f:
            self._chunked_frequency_dict = pkl.load(f)

    def save(self, model_path: os.path):
        """Save the chunked frequency dict inside the given folder.

        Args:
            model_path (os.path): The folder to save the chunked
              frequency dict (pickle file).
        """
        with open(os.path.join(model_path, "model.pkl"), "wb") as f:
            pkl.dump(self._chunked_frequency_dict, f)

    def inference(self, raw_sents: list) -> list:
        """Run the inference on a given list of short texts.

        Raises:
            ValueError: If the model has not yet been trained.

        Args:
            raw_sents (list): The list of raw sents to run the inference on.

        Returns:
            list: The list of documents with predictions.
        """
        if self._chunked_frequency_dict is None:
            raise ValueError(
                "This Dictionary model is not trained yet. "
                "Please run the train function before proceeding."
            )

        preds = []
        
        min_words = min(self._chunked_frequency_dict.keys())
        max_words = max(self._chunked_frequency_dict.keys()) + 1
        cfd = self._chunked_frequency_dict

        for sent in raw_sents:
            tokens = sent
            labels = ["O"] * len(sent)

            for i, t in enumerate(tokens):

                # If label already predicted (by a larger term),
                # move on.
                if labels[i] != "O":
                    continue

                # Go through each number of words (in reverse order).
                # Check each chunk of words to see whether they are in
                # the cfd. If so, set the labels accordingly.
                for j in reversed(range(min_words, max_words)):
                    if j not in cfd:
                        continue
                    if (i + j) > len(tokens):
                        continue
                    token_str = " ".join(tokens[i : i + j])

                    if token_str in cfd[j]:
                        base_class = cfd[j][token_str]
                        labels[i] = "B-" + base_class
                        for x in range((i + 1), (i + j)):
                            labels[x] = "I-" + base_class                            
                            i+= 1 # Skip ahead so the labels are not overwritten
                        

            # Create a ConllDocument from these tokens and labels and append.
            doc = {"tokens": tokens, "labels": labels}
            preds.append(doc)

        return preds

    def _load_conll_data(self, datasets_path: str) -> dict:
        """Load the CONLL-formatted data from the given folder.
        Only loads train and dev, as loading test would give it an unfair
        advantage vs other models.

        Args:
            datasets_path (str): The folder containing the three
              CONLL-formatted files (train.txt, dev.txt, test.txt)

        Returns:
            list: A list of all documents in the training and dev sets. Each doc is
            represented as a dict, i.e.
            {tokens: [list of tokens], labels: [list of labels])}.
        """
        conll_dataset = []
        for ds_name in ["train", "dev"]:
            ds = load_conll_dataset(os.path.join(datasets_path, f"{ds_name}.txt"))
            for doc in ds:
                conll_dataset.append(doc)
        return conll_dataset

    def _chunk_frequency_dict(self, frequency_dict: dict) -> dict:
        """Chunk the frequency dict, i.e. split it into a dict where each
        key is the number of words in each phrase, and then each item in that
        key is the most commonly occurring label for that word, e.g.
        1: {
          "pump": "Item"
        },
        2: {
          "big pump": "Item"
        }

        Args:
            frequency_dict (dict): The non-chunked frequency dict.
        """
        _chunked_frequency_dict = {}
        for (phrase, label_freqs) in frequency_dict.items():
            num_words = len(phrase.split(" "))
            label = max(label_freqs, key=label_freqs.get)
            if num_words not in _chunked_frequency_dict:
                _chunked_frequency_dict[num_words] = {}
            _chunked_frequency_dict[num_words].update({phrase: label})

        return _chunked_frequency_dict

    def _build_frequency_dict(self, conll_dataset: list):
        """Build a dictionary of the frequency of each token mapping to
        each label in the given Redcoat dataset.

        Args:
            conll_dataset (list): The Conll dataset to build the
              dict from.

        Returns:
            dict: A dict mapping each entity mention to a dict of {type:
              frequency}.
        """
        frequency_dict = {}

        for doc in conll_dataset:
            phrase_labels = _get_phrase_labels(doc)
            for (phrase, label) in phrase_labels:
                phrase_str = " ".join(phrase)

                if phrase_str not in frequency_dict:
                    frequency_dict[phrase_str] = {}
                if label not in frequency_dict[phrase_str]:
                    frequency_dict[phrase_str][label] = 0
                frequency_dict[phrase_str][label] += 1

        return frequency_dict

    
def _get_phrase_labels(doc: dict):
    """Return a list of (phrases, labels) for each mention
    in a doc (which is a dict of {"tokens": [tokens in the doc], "labels": [labels of the doc]}.
    Each phrase is a list of words of that label, i.e.
    [['centrifugal', 'pump'], 'Item']

    Returns:
        list: A list of (phrase, labels).
    """
    phrase_labels = []
    current_phrase = []
    for i, (token, label) in enumerate(zip(doc["tokens"], doc["labels"])):
        
        if label.startswith("B-"):
            if len(current_phrase) > 0:
                phrase_labels.append((current_phrase, current_label))
            current_phrase = [token]
            current_label = label[2:]
        elif label.startswith("I-"):
            current_phrase.append(token)
        elif label == "O":
            if len(current_phrase) > 0:
                phrase_labels.append((current_phrase, current_label))
            current_phrase = []
            current_label = None
        if (
            i == len(doc["tokens"]) - 1
            and label != "O"
            and len(current_phrase) > 0
        ):
            phrase_labels.append((current_phrase, current_label))
            
    return phrase_labels  
    

## 2.4. Running inference on unseen sentences

The next step is to use our trained model to infer the entity type of each entity appearing in a list of previously unseen data.

In [151]:
NER_MODELS_PATH = "models/ner_models"


# If you are using the DictionaryNERModel:
m = DictionaryNERModel2183()

# If you are using the FlairNERModel, uncomment this and comment the line above:
#m = FlairNERModel()

m.train(NER_DATASETS_PATH, os.path.join(NER_MODELS_PATH))

tagged_bio_sents = []

sentences = []
for row in work_order_data:
    sentences.append(row["ShortText"].split()) # We must 'tokenise' the sentence first, i.e. split into words

tagged_bio_sents = m.inference(sentences)
    
print(tagged_bio_sents[12])

Building dictionary...
Loaded 3200 documents from data/ner_dataset\train.txt.
Loaded 401 documents from data/ner_dataset\dev.txt.
{'tokens': ['a/c', 'not', 'working'], 'labels': ['B-Item', 'B-Observation', 'I-Observation']}


## 2.5 Converting the BIO format to the "Mention"-based format

The BIO-based format above has one key downside - it is not good for representing 'phrases' of more than one token in length. This makes it difficult to work with for future steps, such as constructing nodes from the entities and running relation extraction. In light of this, we will now convert the BIO-formatted predictions into Mention format, i.e. go from this:

    {'tokens': ['a/c', 'not', 'working'],
     'labels': ['B-Item', 'B-Observation', 'I-Observation']}
    
To this:

    {'tokens': ['a/c', 'not', 'working'],
     'mentions': [
         {'start': 0, 'labels': ['Item'], 'end': 1},
         {'start': 1, 'labels': ['Observation'], 'end': 3}]}
    
Note that this format is also able to now support multiple labels per mention (though we will only be using single labels for simplicity).

This step is just a bit of data wrangling - here we have defined a helper function to convert a BIO-tagged sentence into a Mention-tagged sentence.

In [153]:
def _bio_to_mention(conll_doc: dict):
    """Return a Mention-format representation of a BIO-formatted
    tagged sentence.

    Args:
        conll_doc (ConllDocument): The doc to convert to redcoat.
        doc_idx (int): The id of the doc, necessary to create a Redcoat doc.

    Returns:
        dict: A mention-formatted dict created from the conll_doc.
    """
    tokens = conll_doc["tokens"]
    labels = conll_doc["labels"]
    mentions_list = []

    start = 0
    end = 0
    label = None
    for i, (token, label) in enumerate(
        zip(tokens, labels)
    ):
        if label.startswith("B-"):
            if len(mentions_list) > 0:
                mentions_list[-1]["end"] = i
            mentions_list.append({"start": i, "labels": [label[2:]]})
        elif label == "O" and len(mentions_list) > 0:
            mentions_list[-1]["end"] = i
        if len(mentions_list) == 0:
            continue
        if i == (len(tokens) - 1) and "end" not in mentions_list[-1]:
            mentions_list[-1]["end"] = i + 1
    return {'tokens': tokens, 'mentions': mentions_list}


# For each BIO tagged sentence in tagged_sents, convert it to the mention-based
# representation
tagged_sents_m = []
for doc in tagged_sents:
    mention_doc = _bio_to_mention(doc)
    tagged_sents_m.append(mention_doc)
 
print(tagged_sents_m[12])

{'tokens': ['a/c', 'not', 'working'], 'mentions': [{'start': 0, 'labels': ['Item'], 'end': 1}, {'start': 1, 'labels': ['Observation'], 'end': 3}]}


# 3. Extracting relations between the entities via Relation Extraction

We have extracted the entities appearing in each work order. The next step is to extract the relationships between those entities. We can do this using Relation Extraction.

![alt text](images/building-relations.png "Building relations")

TODO: Flair-based RE model code from MWO2KG rebuild. Very similar to the section above, just with a different dataset and using text classification instead of sequence tagging

# 4. Combining NER+RE

TODO: Do something like this but using RE instead of simple Item -> everything

In [154]:

# This is old code from the master class

triples = []

for row in normalised_work_order_entities:
    for (ngram, entity_class) in row:
        if entity_class != "item": continue
            
        # If this entity is an item, link it to all other entities in the work order       
             
        for (other_ngram, other_entity_class) in row:   
            if ngram == other_ngram: continue # Don't link items to themselves                

            relation_type = other_entity_class.upper()                
            triples.append(((ngram, entity_class), "HAS_%s" % relation_type, (other_ngram, other_entity_class)))
        
for triple in triples:
    print(triple)

NameError: name 'normalised_work_order_entities' is not defined

# 4. Creating the graph

Now that we have our nodes and relations we can go ahead and build the Neo4J graph.

To do this we are going to use py2neo, a Python library for interacting with Neo4J.

There are also a couple of other ways to do this - you can either use Neo4J and run Cypher queries to insert each node and relation, or use the APOC library to import a list of nodes from a CSV file. I find Python to be the simplest way, however.

> Before proceeding, make sure you have created a new graph in Neo4j and that your new Neo4j graph is running.

You can download and install Neo4j from here if you haven't already: https://neo4j.com/download/. I will be demonstrating the graph during the class so there's no need to have it installed unless you are also interested in trying out some graph queries yourself.

> If you need to build your graph again, make sure to run this cell before running subsequent cells.

In [78]:
from py2neo import Graph
from py2neo.data import Node, Relationship

GRAPH_PASSWORD = "password" # Set this to the password of your Neo4J graph

graph = Graph(password = GRAPH_PASSWORD)

# We will start by deleting all nodes and edges in the current graph.
# If we don't do this, we will end up with duplicate nodes and edges when running this script again.
graph.delete_all() 

tx = graph.begin()

# We will keep a dictionary of nodes that we have created so far.
# This serves two purposes:
#  - prevents duplicate nodes
#  - provides us with a way to create edges between the nodes
created_entity_nodes = {}

# Creates a node for the specified ngram and entity_class.
# If the node has already been created (i.e. it exists in created_nodes), return the node.
# Otherwise, create a new one.
def create_entity_node(ngram, entity_class):
    if ngram in created_entity_nodes:
        node = created_entity_nodes[ngram]
    else:
        node = Node("Entity", entity_class, name=ngram)
        created_entity_nodes[ngram] = node
        tx.create(node)
    return node


# Create a node for each triple in the list of triples.
# Set the class of each node to the entity_class (e.g. "activity", "item" or "observation").
# Create a relationship between the nodes in the triple.
for ((ngram_1, entity_class_1), relation, (ngram_2, entity_class_2)) in triples:
    
    node_1 = create_entity_node(ngram_1, entity_class_1)
    node_2 = create_entity_node(ngram_2, entity_class_2)   
    
    
    # Create a relationship between two nodes.
    # This does not check for duplicate relationships unlike create_node,
    # so this code will need to be adjusted on larger datasets.
    relationship = Relationship( node_1, relation, node_2 )
    tx.create(relationship)
    
    
tx.commit()
        

## 4.1 Create nodes for the documents (i.e. the Work Orders)

In order to query our graph, we need to create nodes for each work order in our dataset as well. We then need to link each Document node to every Entity node appearing in that document.

In [79]:
from dateutil.parser import parse as parse_date

# Our work_order_data and normalised_work_order entities allow us to do this quite easily,

tx = graph.begin()

# We will once again keep a mapping of created work order nodes, this time indexed by the row index.
created_work_order_nodes = {}

# Dates are a little awkward in Neo4j - we have to convert it to an integer representation in Python.
# The APOC library has functions to handle this better.
def date_to_int(date):
    parsed_date = parse_date(str(date))
    date = int("%s%s%s" % (parsed_date.year, str(parsed_date.month).zfill(2), str(parsed_date.day).zfill(2)))
    return date

# The process of creating a work order node is a bit different to creating an entity,
# as we also want to incorporate some of the structured fields onto the node.
def create_structured_node(index, row, node_type, created_nodes):
    if index in created_nodes:
        return created_nodes[index]

    if 'StartDate' in row:
        row['StartDate'] = date_to_int(row['StartDate'])
    if 'EndDate' in row:
        row['EndDate'] = date_to_int(row['EndDate'])  

    node = Node(node_type, **row)
    created_nodes[index] = node
    tx.create(node)
    return node

for i, row in enumerate(work_order_data):
    node = create_structured_node(i, row, "WorkOrder", created_work_order_nodes)
    
tx.commit()





## 4.2 Link the entities to their corresponding work order nodes

In order to properly query our graph, we need to link every entity node to the work order node in which it appears.

This allows us to run queries such as "pumps with electrical issues in the last 3 months".

In [80]:
tx = graph.begin()

# We can use the normalised_work_order_entries list to do this.
for i, row in enumerate(normalised_work_order_entities):
    for (ngram, entity_class) in row:        
        
        node_1 = created_entity_nodes[ngram]
        node_2 = created_work_order_nodes[i]
        
        relationship = Relationship( node_1, "APPEARS_IN", node_2 )
        tx.create(relationship)
       
tx.commit()

# 5. Querying the graph

## TODO: Update with GQVis

Now that the graph has been created, we can query it in Neo4j. This section lists some example queries that we can run on our graph. If you would like to try these yourself you can paste them directly into the Neo4j console.

First, let's try a simple query. Here is a query that searches for __all failure modes observed on engines__:

    MATCH (e:Entity {name: "engine"})-[r:HAS_OBSERVATION]->(o:observation)
    RETURN e, r, o

We can also use our graph as a way to quickly search and access work orders for the entities appearing in those work orders. For example, searching for __all work orders containing a leak__:

    MATCH (d:WorkOrder)<-[a:APPEARS_IN]-(o:observation {name: "leak"})
    RETURN d, a, o

We could extend this to also show the items on which the leaks were present:

    MATCH (d:WorkOrder)<-[a:APPEARS_IN]-(o:observation {name: "leak"})<-[r:HAS_OBSERVATION]-(e:Entity)
    RETURN d, a, o, r, e

Our queries can also incorporate structured data, such as the start dates of the work orders. Here is an example query for __all assets that had leaks from 25 to 28 July__:

    MATCH (d:WorkOrder)<-[a:APPEARS_IN]-(e:Entity)-[r:HAS_OBSERVATION]->(o:observation {name: "leak"})-[:APPEARS_IN]->(d)
    WHERE d.StartDate >= 20050725
    AND d.StartDate <= 20050728
    RETURN e, r, o

On a larger graph this would also work well with other forms of structured data such as costs. We could query based on specific asset costs, for example.

Now that our work orders and downtime events are in one graph, we can also make queries about downtime events. Here is an example query for the __downtime events associated with assets appearing in work orders from 25 to 28 July (where the downtime events occurred in July)__:

    MATCH (d:WorkOrder)<-[a:APPEARS_IN]-(e:Entity)-[r:HAS_EVENT]->(x:DowntimeEvent)
    WHERE d.StartDate > 20050725
    AND d.StartDate < 20050728
    AND 20050700 <= x.StartDate <= 20050731
    RETURN e, r, x

We can of course extend this to specific assets, such as pumps:

    MATCH (d:WorkOrder)<-[a:APPEARS_IN]-(e:Entity {name: "pump"})-[r:HAS_EVENT]->(x:DowntimeEvent)
    WHERE d.StartDate > 20050725
    AND d.StartDate < 20050728
    AND 20050700 <= x.StartDate <= 20050731
    RETURN e, r, x

In larger graphs the downtime events could even be further queried based on duration, cost, lost feed, or date ranges.

# 6. Future improvements

## Incorporating FLOCs

Our downtime events are currently linked to Item nodes, but it would make more sense to link them to nodes representing the functional locations.

If you are interested in continuing work on this small graph, the next best step would be to create nodes for the functional location data (`floc_data`) and to link the downtime events to those nodes as opposed to the Item nodes.

![alt text](images/adding-flocs.png "Adding FLOCs")

## Frequencies on edge properties

We could also improve the graph by incorporating frequencies onto the edge properties. For example, if a "leak" occurred on a pump in two different work orders, our link between "pump" and "leak" could have a property called `frequency` with a value of `2`. This would allow us to query, for example, assets that had a particularly high number of leaks.


## Constructing a graph from your own work order data

If you have a work order dataset of your own, feel free to download this code and try it out on your dataset. I would be happy to chat if you would like to further discuss the code or if you run into any issues.

If you need to extract entities not listed in the lexicon, you will need to update the lexicon file to include your new entities. Alternatively, the LexiconTagger can be substituted for a named entity recognition model.

In [82]:
floc_file = "data/sample_flocs.csv"
floc_data = load_csv(floc_file)

# Your code here

# ---------------------------------------------------------------------------------------

## OLD Normalise the entities

### MS: Probably going to take this out to save time, I can leave it as a future step for people. It should probably go before NER/RE anyway

The next step is to normalise the ngrams, i.e. convert each ngram into a normalised form. This is important as we would prefer to have a single node for a single concept, e.g. one node for "engine" as opposed to two nodes for "engin" and "engine".

We will once again be using a lexicon for this task, but it would typically be performed by machine learning.

![alt text](images/normalising-entities.png "Normalising entities")

In [76]:
lexicon_n_file = "data/lexicon_normalisation.csv"
lexicon_normaliser = LexiconTagger(lexicon_n_file)

normalised_work_order_entities = []

# For every row in work_order_entities, replace each ngram with its normalised counterpart
# as per the normalisation lexicon.
# For example, "engin" will become "engine", "leaking" will become "leak", etc.
for row in work_order_entities:
    normalised_work_order_entities.append([(lexicon_normaliser.normalise_ngram(ngram), entity_class) 
                                           for (ngram, entity_class) in row])
    
    
for row in normalised_work_order_entities:
    print(row)

[('repair', 'activity'), ('cracked', 'observation'), ('hydraulic tank', 'item')]
[('engine', 'item'), ('failure to start', 'observation')]
[('air conditioner', 'item'), ('overheating', 'observation')]
[('engine', 'item'), ('breakdown', 'observation')]
[('fix', 'activity'), ('engine', 'item')]
[('pump', 'item'), ('service', 'activity')]
[('pump', 'item'), ('leak', 'observation')]
[('fix', 'activity'), ('leak', 'observation'), ('pump', 'item')]
[('engine', 'item'), ('breakdown', 'observation')]
[('engine', 'item'), ('failure to start', 'observation')]
[('pump', 'item'), ('electrical issue', 'observation')]
[('pump', 'item'), ('leak', 'observation')]
[('air conditioner', 'item'), ('breakdown', 'observation')]
[('air conditioner', 'item'), ('breakdown', 'observation')]


## OLD Extending the graph to incorporate Downtime events

The next step is to incorporate the downtime events.

For this exercise we are going to link the Downtime events to the first Item node appearing in the work orders with the same FLOC as the downtime event.


![alt text](images/adding-downtime-events.png "Adding downtime events")

In [81]:
tx = graph.begin()

created_downtime_nodes = {}

# Create a DowntimeEvent node for each row
for i, downtime_row in enumerate(downtime_data):
    node = create_structured_node(i, downtime_row, "DowntimeEvent", created_downtime_nodes)
    
    # Get all work order nodes with the same FLOC and link the DowntimeEvent to the Items appearing
    # in those work orders
    for j, work_order_row in enumerate(work_order_data):
        if work_order_row["FLOC"] == downtime_row["FLOC"]:
            
            work_order_entities = normalised_work_order_entities[j]
            
            for (ngram, entity_class) in work_order_entities:
                if entity_class != "item": continue    # We don't need to link non-items to downtime events               
                    
                item_node = created_entity_nodes[ngram]
                relationship = Relationship( item_node, "HAS_EVENT", node )
                tx.create(relationship)
                break

    
tx.commit()
